In [8]:
import pandas as pd

url = ('/Users/javi/Desktop/Ironhack/Entregas de github/Personal-Project-at-Ironhack/global_power_plant_database_v_1_3/global_power_plant_database.csv')

df = pd.read_csv(url)

/var/folders/23/ykjhg9v57tn3wfmhgxtgj1s80000gn/T/ipykernel_57063/3900116404.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


In [9]:
df.shape

(34936, 36)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34936 entries, 0 to 34935
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country                         34936 non-null  object 
 1   country_long                    34936 non-null  object 
 2   name                            34936 non-null  object 
 3   gppd_idnr                       34936 non-null  object 
 4   capacity_mw                     34936 non-null  float64
 5   latitude                        34936 non-null  float64
 6   longitude                       34936 non-null  float64
 7   primary_fuel                    34936 non-null  object 
 8   other_fuel1                     1944 non-null   object 
 9   other_fuel2                     276 non-null    object 
 10  other_fuel3                     92 non-null     object 
 11  commissioning_year              17447 non-null  float64
 12  owner                           

After some initial exploration of the df of world wide powerplant list, I decided to use either estimated_energy generation of 2017 or the actual number (df columns generation_gwh_2017 or  estimated_generation_note_2017). The problem with the actual numner of 2017 is that we only have around 9500 valid entries world wide. And that would mean getting rid of 70% if the available data. With the use of estimated energy prodcution where in most cases it´s an overestimation. but not by a lot...
I´ll do some major data cleaning. getting rid on most of the columns. The ones i´ll keep is country, contry_long, name, capacity_mw,latitude, longitude, primeray fuel, owner, estimated_generation_gwh_2017. 
After cleaning all I can and using fillna when necessary. or dropna to. I´ll add another column to this column with data I have webscraped from the internet which should calculate the amount of co2 produced by the estimated energy production. after that I'll also add another column which should calculate the estimated carbon tax on this 
another column... if i can find with either webscraping or selenium... the average price of 2017 

In [11]:
#confirmation if estimation is worth or accurate. I have considered that it is accurate enough to pull from there 
plt.figure(figsize=(10, 6))
sns.boxplot(data=df[['generation_gwh_2017', 'estimated_generation_gwh_2017']])
plt.title('Box Plot of Generation vs. Estimated Generation')
plt.xlabel('Variable')
plt.ylabel('Value')
plt.show()

#scatter plot offers quite insufficient data. At least. I don´t know how I can make good use of this information
plt.figure(figsize=(8, 6))
sns.scatterplot(x='generation_gwh_2017', y='estimated_generation_gwh_2017', data=df)
plt.title('Scatter Plot of Generation vs. Estimated Generation')
plt.xlabel('Generation GWh 2017')
plt.ylabel('Estimated Generation GWh 2017')
plt.show()


NameError: name 'plt' is not defined

In [12]:
#here i had to get rid of the Nuclear which was the biggest contributor 
cleaned_df = df.dropna(subset=['generation_gwh_2017', 'estimated_generation_gwh_2017'])

# Exclude rows with 'Nuclear' primary fuel
cleaned_df = cleaned_df[cleaned_df['primary_fuel'] != 'Nuclear']

# Group by the 'primary_fuel' column and calculate the mean for both actual and estimated generation values
grouped_data = cleaned_df.groupby('primary_fuel').agg({'generation_gwh_2017': 'mean', 'estimated_generation_gwh_2017': 'mean'}).reset_index()

# Sort the DataFrame by 'primary_fuel' column
grouped_data = grouped_data.sort_values(by='primary_fuel')

# Plot the bar plot
plt.figure(figsize=(12, 6))
sns.barplot(x='primary_fuel', y='generation_gwh_2017', data=grouped_data, color='blue', label='Actual Generation')
sns.barplot(x='primary_fuel', y='estimated_generation_gwh_2017', data=grouped_data, color='red', alpha=0.5, label='Estimated Generation')
plt.title('Comparison of Actual and Estimated Generation by Primary Fuel Type (Excluding Nuclear)')
plt.xlabel('Primary Fuel Type')
plt.ylabel('Generation GWh 2017')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.legend()
plt.tight_layout()  # Adjust layout to prevent overlap of labels
plt.show()


NameError: name 'plt' is not defined

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34936 entries, 0 to 34935
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country                         34936 non-null  object 
 1   country_long                    34936 non-null  object 
 2   name                            34936 non-null  object 
 3   gppd_idnr                       34936 non-null  object 
 4   capacity_mw                     34936 non-null  float64
 5   latitude                        34936 non-null  float64
 6   longitude                       34936 non-null  float64
 7   primary_fuel                    34936 non-null  object 
 8   other_fuel1                     1944 non-null   object 
 9   other_fuel2                     276 non-null    object 
 10  other_fuel3                     92 non-null     object 
 11  commissioning_year              17447 non-null  float64
 12  owner                           

In [14]:
columns_to_drop = ['gppd_idnr','other_fuel1','commissioning_year','owner','latitude','longitude', 'source','url', 'geolocation_source', 'wepp_id', 'year_of_capacity_data', 'other_fuel2', 'other_fuel3','generation_gwh_2013','generation_gwh_2014','generation_gwh_2015','generation_gwh_2016','generation_gwh_2017','generation_gwh_2018','generation_gwh_2019','generation_data_source','estimated_generation_gwh_2013','estimated_generation_gwh_2014','estimated_generation_gwh_2015','estimated_generation_gwh_2016','estimated_generation_note_2013','estimated_generation_note_2014','estimated_generation_note_2015','estimated_generation_note_2016','estimated_generation_note_2017']
df.drop(columns_to_drop, axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34936 entries, 0 to 34935
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   country                        34936 non-null  object 
 1   country_long                   34936 non-null  object 
 2   name                           34936 non-null  object 
 3   capacity_mw                    34936 non-null  float64
 4   primary_fuel                   34936 non-null  object 
 5   estimated_generation_gwh_2017  33138 non-null  float64
dtypes: float64(2), object(4)
memory usage: 1.6+ MB


In [15]:
df['primary_fuel'].unique()

array(['Hydro', 'Solar', 'Gas', 'Other', 'Oil', 'Wind', 'Nuclear', 'Coal',
       'Waste', 'Biomass', 'Wave and Tidal', 'Petcoke', 'Geothermal',
       'Storage', 'Cogeneration'], dtype=object)

In [22]:
df.isna().sum()

country                             0
country_long                        0
name                                0
capacity_mw                         0
primary_fuel                        0
estimated_generation_gwh_2017    1798
dtype: int64

In [23]:
df.dropna(subset=['estimated_generation_gwh_2017'], inplace=True)
df.isna().sum()

country                          0
country_long                     0
name                             0
capacity_mw                      0
primary_fuel                     0
estimated_generation_gwh_2017    0
dtype: int64

In [25]:
df.shape

(33138, 6)

In [29]:
df.head(50)

,country,country_long,name,capacity_mw,primary_fuel,estimated_generation_gwh_2017
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,33.00,Hydro,119.50
1,AFG,Afghanistan,Kandahar DOG,10.00,Solar,18.29
2,AFG,Afghanistan,Kandahar JOL,10.00,Solar,18.72
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,66.00,Hydro,174.91
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,100.00,Hydro,350.80
5,AFG,Afghanistan,Nangarhar (Darunta) Hydroelectric Power Plant ...,11.55,Hydro,46.12
7,AFG,Afghanistan,Pul-e-Khumri Hydroelectric Power Plant Afghani...,6.00,Hydro,19.74
8,AFG,Afghanistan,Sarobi Dam Hydroelectric Power Plant Afghanistan,22.00,Hydro,80.00
9,ALB,Albania,Bistrica 1,27.00,Hydro,88.45
10,ALB,Albania,Fierza,500.00,Hydro,1648.24


In [30]:
df['primary_fuel'].unique()

array(['Hydro', 'Solar', 'Gas', 'Oil', 'Other', 'Nuclear', 'Coal', 'Wind',
       'Waste', 'Geothermal'], dtype=object)

In [31]:
df.to_csv('COUNTRIES_CLEANED.csv', index=False)